In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

In [2]:
with open('.env') as f:
    content = f.readlines()

import os

for line in content:
    if 'OPENAI_KEY' in line:
        key, value = line.strip().split('=', 1)
        os.environ[key] = value

OPENAI_KEY = os.getenv('OPENAI_KEY')

In [3]:
client = OpenAI(api_key=OPENAI_KEY)

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

In [6]:
es_client.info()

ObjectApiResponse({'name': 'c91feff7d39f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ynhtqtUUTH6TxZdcVOnMyw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

In [7]:
!curl localhost:9200

{
  "name" : "c91feff7d39f",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "ynhtqtUUTH6TxZdcVOnMyw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0  70958      0 --:--:-- --:--:-- --:--:-- 77000


In [61]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


documents[:2]        

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

# Q2. Indexing the data

Which function do you use for adding your data to elastic?

- insert
- ***index***
- put
- add


In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

# Q3. Searching

Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

- 94.05
- ***84.05***
- 74.05
- 64.05


In [11]:
query = 'How do I execute a command in a running docker container?'

In [12]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_score'])

C:\Users\JhonJairo\AppData\Local\Temp\ipykernel_20712\1662656835.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [13]:
result_docs

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

# Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?
- ***How do I copy files from a different folder into docker container’s working directory?***
- How do Lambda container images work?
- How can I annotate a graph?


In [33]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source']) 

C:\Users\JhonJairo\AppData\Local\Temp\ipykernel_20712\1016376780.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [34]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

# Q5. Building a prompt

What's the length of the resulting prompt? (use the len function)

- 962
- ***1462***
- 1962
- 2462


In [35]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs     

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [37]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
prompt.split('\n\n')


C:\Users\JhonJairo\AppData\Local\Temp\ipykernel_20712\2205967610.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


["You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.",
 'QUESTION: How do I execute a command in a running docker container?',
 'CONTEXT: \nsection: 5. Deploying Machine Learning Models\nquestion: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
 "section: 5. Deploying Machine Learning Models\nquestion: How do I copy files from my local machine to docker container?\nanswer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Doc

In [38]:
len(prompt)

1637

# Q6. Tokens

Use the encode function. How many tokens does our prompt have?

- 122
- 222
- ***322***
- 422 

In [39]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [53]:
# Encode the prompt to get the number of tokens
tokens_promt = encoding.encode(prompt)
num_tokens_promt = len(tokens_promt)

print(f"The number of tokens in the prompt is: {num_tokens_promt}")

The number of tokens in the prompt is: 356


# Bonus: generating the answer (ungraded)

In [41]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [42]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [44]:
response = rag(query)

C:\Users\JhonJairo\AppData\Local\Temp\ipykernel_20712\2205967610.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [48]:
response.split('\n')

['To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps:',
 '',
 '1. First, list the running containers to find the container ID using the command:',
 '    ```sh',
 '    docker ps',
 '    ```',
 '',
 '2. Once you have the container ID, you can execute a command in the specific container. For example, to start a bash session, use:',
 '    ```sh',
 '    docker exec -it <container-id> bash',
 '    ```']

# Bonus: calculating the costs (ungraded)

In [54]:
tokens_response = encoding.encode(response)
num_tokens_response = len(tokens_response)

print(f"The number of tokens in the response is: {num_tokens_response}")

The number of tokens in the response is: 100


In [56]:
input_cost = num_tokens_promt * (0.005 / 1000)
output_cost = num_tokens_response * (0.015 / 1000)

In [60]:
print(f"The total costs for 1000 requests are: ${1000 * (input_cost + output_cost)}")

The total costs for 1000 requests are: $3.28
